# Introduction

This program performs analysis and visualization of openLCA results from an LCIA calculation, starting with an excel file export. This program assumes that you've already compiled an excel file of the unit processes exported from your openLCA database, with ISIC categories assigned to each process. This excel file should be named "database_process_summary_added.xlsx" with the corresponding file path.

# Setup

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Set up variables
# Change these variables to match your data

impact_category_name = "climate change - global warming potential (GWP100) (Nitrous Oxide, Methane, Carbon Dioxide)"
product_system_name = "GeoH2 (85% mol H2; H2 Average Fugitive; CH4 Capture)"


# LCA results file path and sheet names
lca_results_file_path = r'C:\Users\laura\OneDrive\Documents\UT_graduate\WEG_graduate_research\Hydrogen_fugitive_emissions_natural_h2\openLCA_ecoinvent\Hydrogen_gas__at_processing__production_mixture__to_consumer__kg___US__85__mol_H2____H2_Average___CH4_Capture.xlsx'
direct_impact_sheet_name = 'Direct impact contributions' #sheet name in the LCA results file
impact_by_flow_sheet_name = 'Impact contributions by flow' #sheet name in the LCA results file

# Database process summary file path
database_process_summary_file_path = r'C:\Users\laura\OneDrive\Documents\UT_graduate\WEG_graduate_research\Hydrogen_fugitive_emissions_natural_h2\openLCA_ecoinvent\openLCA\database_process_summary_added.xlsx'

# Define the output Excel file path
output_excel_file_path = r'C:\Users\laura\OneDrive\Documents\UT_graduate\WEG_graduate_research\Hydrogen_fugitive_emissions_natural_h2\openLCA_ecoinvent\openLCA' + f'\{product_system_name}_data_export.xlsx'

<string>:17: SyntaxWarning: invalid escape sequence '\{'
<>:17: SyntaxWarning: invalid escape sequence '\{'
<string>:17: SyntaxWarning: invalid escape sequence '\{'
<>:17: SyntaxWarning: invalid escape sequence '\{'
C:\Users\laura\AppData\Local\Temp\ipykernel_34388\3916949549.py:17: SyntaxWarning: invalid escape sequence '\{'
  output_excel_file_path = r'C:\Users\laura\OneDrive\Documents\UT_graduate\WEG_graduate_research\Hydrogen_fugitive_emissions_natural_h2\openLCA_ecoinvent\openLCA' + f'\{product_system_name}_data_export.xlsx'


# Read in LCA Results Excel File
Read in LCA results impact by unit process and by flow

In [3]:
# Read the specified worksheet into a DataFrame
df_process_impact = pd.read_excel(lca_results_file_path, sheet_name=direct_impact_sheet_name)
df_flows_impact = pd.read_excel(lca_results_file_path, sheet_name=impact_by_flow_sheet_name)

In [4]:
# Drop the first column if it contains only empty values
if df_process_impact.iloc[:, 0].isnull().all():
    df_process_impact.drop(df_process_impact.columns[0], axis=1, inplace=True)
if df_flows_impact.iloc[:, 0].isnull().all():
    df_flows_impact.drop(df_flows_impact.columns[0], axis=1, inplace=True)

# Transpose the DataFrame
df_process_impact_transposed = df_process_impact.transpose()
df_flows_impact_transposed = df_flows_impact.transpose()

# Reset the index of the transposed DataFrame
df_process_impact_transposed.reset_index(drop=True, inplace=True)
df_flows_impact_transposed.reset_index(drop=True, inplace=True)

In [5]:
# Find the column index of the target impact category in the transposed DataFrames
process_column_index = df_process_impact_transposed.columns[df_process_impact_transposed.iloc[1] == impact_category_name].tolist()
flows_column_index = df_flows_impact_transposed.columns[df_flows_impact_transposed.iloc[1] == impact_category_name].tolist()

# If the column is found, retrieve its index; otherwise, return None
process_column_index = process_column_index[0] if process_column_index else None
flows_column_index = flows_column_index[0] if flows_column_index else None
print(f"Process column index: {process_column_index}")
print(f"Flows column index: {flows_column_index}")

# Create a copy of transposed DataFrames with only specified columns
process_columns_to_keep = [0, 1, 2, process_column_index]
flows_columns_to_keep = [0, 1, 2, 3, 4, flows_column_index]
df_process_impact_filtered = df_process_impact_transposed.iloc[:, process_columns_to_keep]
df_flows_impact_filtered = df_flows_impact_transposed.iloc[:, flows_columns_to_keep]

impact_category_uuid = df_process_impact_filtered.iloc[0, 3]
impact_category_reference_unit = df_process_impact_filtered.iloc[2, 3]
print(f"Impact category UUID: {impact_category_uuid}")
print(f"Impact category reference unit: {impact_category_reference_unit}")

# Remove unnecessary rows and rename the columns of the filtered DataFrames
df_process_impact_filtered = df_process_impact_filtered.iloc[4:].reset_index(drop=True)
df_process_impact_filtered.columns = ['Process_UUID', 'Process_Name', 'Location', 'GWP100_Impact_Value_kg_CO2eq']
df_flows_impact_filtered = df_flows_impact_filtered.iloc[4:].reset_index(drop=True)
df_flows_impact_filtered.columns = ['Flow_UUID', 'Flow_Name', 'Flow_Category', 'Flow_Sub-category', 'Flow_Unit', 'GWP100_Impact_Value_kg_CO2eq']


Process column index: 7
Flows column index: 9
Impact category UUID: f1701a93-f252-4082-92f8-390ad9fd9bae
Impact category reference unit: kg CO2-Eq


# Read in OpenLCA Database Process Summary Excel File with ISIC Classifications
Read in unique list of unit processes in openLCA database

In [6]:
# Read in the database process summary file
database_process_df = pd.read_excel(database_process_summary_file_path)

In [7]:
# Split column "ISIC_Category" into multiple columns using "/" as the delimiter
split_columns = database_process_df['ISIC_Category'].str.split('/', expand=True)

# Rename the new columns for clarity
split_columns.columns = ['ISIC_Category_Section', 'ISIC_Category_Division', 'ISIC_Category_Group', 'ISIC_Category_Class']

# Add the split columns as additional columns to database_process_df
database_process_df = pd.concat([database_process_df, split_columns], axis=1)

# Merge Database Process Summary DataFrame to LCA results Process Impact DataFrame

In [8]:
# Merge the database process summary DataFrame with the filtered process impact DataFrame
merged_process_df = pd.merge(df_process_impact_filtered, database_process_df, on='Process_UUID', how='left')

# Check if the "Process_Name" column exists in both DataFrames after the merge
if 'Process_Name_x' in merged_process_df.columns and 'Process_Name_y' in merged_process_df.columns:
    # Check if the "Process_Name" columns match in both DataFrames
    if (merged_process_df['Process_Name_x'] == merged_process_df['Process_Name_y']).all():
        # Drop one of the "Process_Name" columns and rename the other to "Process_Name"
        merged_process_df.drop(columns=['Process_Name_y'], inplace=True)
        merged_process_df.rename(columns={'Process_Name_x': 'Process_Name'}, inplace=True)
    else:
        print("Warning: Process_Name columns do not match completely.")
else:
    print("Error: Process_Name columns are missing in the merged DataFrame.")

# Analyze DataFrames

In [9]:
# Convert the GWP100 impact value to numeric, forcing errors to NaN
merged_process_df['GWP100_Impact_Value_kg_CO2eq'] = pd.to_numeric(merged_process_df['GWP100_Impact_Value_kg_CO2eq'], errors='coerce')
df_process_impact_filtered['GWP100_Impact_Value_kg_CO2eq'] = pd.to_numeric(df_process_impact_filtered['GWP100_Impact_Value_kg_CO2eq'], errors='coerce')

# Drop rows with NaN values in the GWP100 impact value column
merged_process_df.dropna(subset=['GWP100_Impact_Value_kg_CO2eq'], inplace=True)
df_flows_impact_filtered.dropna(subset=['GWP100_Impact_Value_kg_CO2eq'], inplace=True)

In [10]:
# Group the data by "Process_Name" and sum the GWP100 impact values
grouped_process_df = merged_process_df.groupby('Process_Name')['GWP100_Impact_Value_kg_CO2eq'].sum().reset_index()
grouped_process_df.sort_values(by='GWP100_Impact_Value_kg_CO2eq', ascending=False, inplace=True)

# Group the data by "ISIC_Category_Section" and sum the GWP100 impact values
grouped_isic_df = merged_process_df.groupby('ISIC_Category_Section')['GWP100_Impact_Value_kg_CO2eq'].sum().reset_index()
grouped_isic_df.sort_values(by='GWP100_Impact_Value_kg_CO2eq', ascending=False, inplace=True)

# Group the data by "Flow_Name" and sum the GWP100 impact values
grouped_flows_df = df_flows_impact_filtered.groupby('Flow_Name')['GWP100_Impact_Value_kg_CO2eq'].sum().reset_index()
grouped_flows_df.sort_values(by='GWP100_Impact_Value_kg_CO2eq', ascending=False, inplace=True)

In [ ]:
# Write the grouped DataFrames to separate sheets in the Excel file
with pd.ExcelWriter(output_excel_file_path, engine='xlsxwriter') as writer:
    grouped_process_df.to_excel(writer, sheet_name='Grouped_Process', index=False)
    grouped_isic_df.to_excel(writer, sheet_name='Grouped_ISIC', index=False)
    grouped_flows_df.to_excel(writer, sheet_name='Grouped_Flows', index=False)

print(f"Grouped data has been exported to {output_excel_file_path}")

ModuleNotFoundError: No module named 'xlsxwriter'